<a href="https://colab.research.google.com/github/ank090/hate_speech_detection/blob/main/hate_speech_detection_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing libraries

In [33]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer
import re

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Loading dataset

In [34]:
dataset=pd.read_csv('hate_speech_detection.csv')
tweets=dataset.iloc[:,2].values
test=dataset.iloc[:10,2].values #for checking the working of clean method

Cleaning text

In [35]:
def clean(text):
  STOPWORDS.update({'is','the','of','to','on','so'})
  clean_tweet=[]
  for i in range(len(text)):
    sample=re.sub(r'[^a-zA-Z]'," ",text[i])
    sample=re.sub(r"[.,'\"@#!?]"," ",sample)
    sample=sample.lower()
    sample=sample.split()
    lm=WordNetLemmatizer()
    for word in sample:
      word=lm.lemmatize(word)
    sample=list(set(sample)-STOPWORDS)
    sample=" ".join(sample)
    clean_tweet.append(sample)
  
  return clean_tweet
clean(test)#Showing the 10 clean tweets from dataset 

['run selfish dysfunctional drags dysfunction user father kids',
 'vans disapointed pdx credit thanks don getthanked t lyft cause offer wheelchair use user',
 'majesty bihday',
 'love u take model time ur',
 'society factsguide motivation now',
 'chaos big allshowandnogo talking fare huge leave pay fan disputes',
 'danny user camping tomorrow',
 'think year hate next exams girl t imagine revolutionschool school actorslife',
 'love clevelandcavaliers allin land cavs won cleveland champions',
 's m welcome gr user']

Creating tokens

In [36]:
cv=CountVectorizer(max_features=10000)
x=cv.fit_transform(clean(tweets)).toarray()
y=dataset.iloc[:,1].values

Splitting dataset in to test and training set

In [37]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.3,random_state=0)

Training model on training set

In [38]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=50,criterion='entropy')
rf.fit(xtrain,ytrain)
ypred=rf.predict(xtest)

Checking accuracy

In [39]:
from sklearn.metrics import classification_report
print('Classification Report of the Random Forest model: \n',classification_report(ytest,ypred))

Classification Report of the Random Forest model: 
               precision    recall  f1-score   support

           0       0.82      0.90      0.86       887
           1       0.85      0.75      0.80       686

    accuracy                           0.83      1573
   macro avg       0.84      0.82      0.83      1573
weighted avg       0.83      0.83      0.83      1573

